## Make test data folder
- Make folder with images cropped to 128x128x128
- within that need subfolder petct 
- and subfolder pet_mask
- then will need to adapt to nnunet (think i need nii.gz images and correct naming ie 0000 and 0001)


In [1]:
import os
import torch
from torch.utils.data import Dataset
import random
import numpy as np
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
import pickle
from scipy import ndimage
import argparse
import matplotlib.pyplot as plt

In [16]:
def Random_Crop_128_oversample(sample,random_seed = None):
    # input = sample dictionary with petct npy and label npy, returns another dictionary of numpy arrays that is cropped

    if random_seed is not None:
        np.random.seed(random_seed)
        random.seed(random_seed)
    

    image, label = sample['image'],sample['label']
    Max_C,max_H,max_W,max_D = image.shape

    if np.max(label)>0:
            # if there is some positive regions in the image, should happen 1/2 of time, make sure random cro[ includes pos point
        indices = np.argwhere(label == 1)
        random_index = np.random.choice(len(indices))
        random_point = indices[random_index] # get 4d coordinates - but how do i know they they are at the edges
        C,H,W,D = random_point[0],random_point[1],random_point[2],random_point[3]
        if H + 64 > max_H: # check if point is not at edge of cube
            H = H - 64
        elif H - 64 < 0:
            H = H + 64
        if W + 64 > max_W:
            W = W - 64
        elif W - 64 < 0:
            W = W+64
        if D + 64 > max_D:
            D = D-64
        elif D - 64 < 0:
            D = D+64
            
    else:
        H = random.randint(64, max_H - 64)
        W = random.randint(64, max_W - 64)
        D = random.randint(64, max_D - 64)
    
    image = image[...,H-64: H + 64, W-64: W + 64, D-64: D + 64] 
    label = label[...,H-64: H + 64, W-64: W + 64, D-64: D + 64] 
        
    return {'image': image, 'label': label}

def Center_Crop_128_oversample(sample,random_seed = None):
    # input = sample dictionary with petct npy and label npy, returns another dictionary of numpy arrays that is cropped

    if random_seed is not None:
        np.random.seed(random_seed)
        random.seed(random_seed)
    

    image, label = sample['image'],sample['label']
    Max_C,max_H,max_W,max_D = image.shape
    print(max_H)
    print(max_W)
    print(max_D)

    H = int(np.round(max_H/2))
    W = int(np.round(max_W/2))
    D = int(np.round(max_D/2))
    
    image = image[...,H-64: H + 64, W-64: W + 64, D-64: D + 64] 
    label = label[...,H-64: H + 64, W-64: W + 64, D-64: D + 64] 
        
    return {'image': image, 'label': label}

In [17]:
root = 'train_data'
list_file = 'test_imgs.txt'

cropped_root = 'test_data'

names =[]
with open(list_file) as f:
    for line in f:
        # list of files will have names of all images without file ending
        line = line.strip()
        name = line
        names.append(name)

for name in names:
    label_path = os.path.join(root,'pet_mask',name+'_mask.npy')
    image_path = os.path.join(root,'petct',name+'_petct.npy')
               
    label = np.load(label_path)
    image = np.load(image_path)

    sample = {'image':image,'label':label}

    #cropped_sample = Random_Crop_128_oversample(sample,random_seed=42)
    cropped_sample = Center_Crop_128_oversample(sample,random_seed=42)
    cropped_image = cropped_sample['image']
    cropped_label = cropped_sample['label']
    
    # resave new images in folder (need to make these folders)
    cropped_label_path = os.path.join(cropped_root,'pet_mask',name+'_mask.npy')
    cropped_image_path = os.path.join(cropped_root,'petct',name+'_petct.npy')
    np.save(cropped_image_path,cropped_image)
    np.save(cropped_label_path,cropped_label)

326
400
400
284
400
400
326
400
400
330
400
400
326
400
400
326
400
400
284
400
400
284
400
400
284
400
400
338
400
400
628
400
400
284
400
400
368
400
400
326
400
400
586
400
400
326
400
400
334
400
400
619
400
400
550
400
400
284
400
400
284
400
400
326
400
400
326
400
400
326
400
400
284
400
400
326
400
400
342
400
400
327
400
400
326
400
400
326
400
400
326
400
400
326
400
400
301
400
400
326
400
400
578
400
400
577
400
400
307
400
400
312
400
400
326
400
400
327
400
400
326
400
400
326
400
400
200
400
400
284
400
400
284
400
400
242
400
400
326
400
400
284
400
400
284
400
400
284
400
400
284
400
400
326
400
400
326
400
400
368
400
400
277
400
400
284
400
400
326
400
400
326
400
400
326
400
400
274
400
400
242
400
400
339
400
400
619
400
400
326
400
400
326
400
400
368
400
400
326
400
400
347
400
400
326
400
400
326
400
400
284
400
400
326
400
400
326
400
400
284
400
400
328
400
400
326
400
400
576
400
400
497
400
400
535
400
400
326
400
400
284
400
400
326
400
400
284
400
400
265


In [98]:
for image in os.listdir(os.path.join(cropped_root,'pet_mask')):
    arr = np.load(os.path.join(cropped_root,'pet_mask',image))
    print(arr.shape)

(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128, 128)
(1, 128, 128

In [19]:
print(len(os.listdir(os.path.join(cropped_root,'petct'))))
print(len(os.listdir(os.path.join(cropped_root,'pet_mask'))))


91
91


## Now need to adapt to nnunet
- images need to be in .nii.gz format
- with correct naming (eg 'PETCT_a3ce52b2a8_11152003_0001.nii.gz for PET and 'PETCT_a3ce52b2a8_11152003.nii.gz' for labels)
- will add them to nnUnet_raw in subfolder Dataset222_PETCT, subfolders labelsTs and imagesTs

In [31]:
import nibabel as nib

# save labels 
root = '/gpfs3/well/papiez/users/hri611/python/nnUnet_raw/'
new_dir = 'Dataset222_PETCT'

cropped_root = 'test_data'

mask_folder = os.path.join(cropped_root, 'pet_mask')
mask_files = [f for f in os.listdir(mask_folder) if f.endswith('.npy')]

# Process each .npy file and save it as .nii.gz
for mask_file in mask_files:
    mask_array = np.load(os.path.join(mask_folder, mask_file))[0]

    nii_filename = mask_file.replace('_mask.npy', '.nii.gz')
    nii_filepath = os.path.join(root,new_dir,'labelsTs', nii_filename)

    # Create a NIfTI image from the numpy array
    mask_nii = nib.Nifti1Image(mask_array, affine=np.eye(4))

    # Save the NIfTI image as .nii.gz
    nib.save(mask_nii, nii_filepath)

    print(f"Saved {nii_filename} to {os.path.join(root,new_dir,'labelsTs')}")


Saved PETCT_9902caa7ec_08132006.nii.gz to /gpfs3/well/papiez/users/hri611/python/nnUnet_raw/Dataset222_PETCT/labelsTs
Saved PETCT_ae46202abb_02282003.nii.gz to /gpfs3/well/papiez/users/hri611/python/nnUnet_raw/Dataset222_PETCT/labelsTs
Saved PETCT_bd76c8d299_10252002.nii.gz to /gpfs3/well/papiez/users/hri611/python/nnUnet_raw/Dataset222_PETCT/labelsTs
Saved PETCT_176bdc5388_06222006.nii.gz to /gpfs3/well/papiez/users/hri611/python/nnUnet_raw/Dataset222_PETCT/labelsTs
Saved PETCT_b621742469_11042004.nii.gz to /gpfs3/well/papiez/users/hri611/python/nnUnet_raw/Dataset222_PETCT/labelsTs
Saved PETCT_d626611daf_04032003.nii.gz to /gpfs3/well/papiez/users/hri611/python/nnUnet_raw/Dataset222_PETCT/labelsTs
Saved PETCT_9c36b318e8_11042006.nii.gz to /gpfs3/well/papiez/users/hri611/python/nnUnet_raw/Dataset222_PETCT/labelsTs
Saved PETCT_44b95650c3_10022003.nii.gz to /gpfs3/well/papiez/users/hri611/python/nnUnet_raw/Dataset222_PETCT/labelsTs
Saved PETCT_14929994cf_04302005.nii.gz to /gpfs3/well/pa

In [30]:
test = nib.load('/gpfs3/well/papiez/users/hri611/python/nnUnet_raw/Dataset220_PETCT/imagesTs/PETCT_a3ce52b2a8_11152003_0001.nii.gz')
test.shape

(400, 400, 326)

In [25]:
# now do the same for petct images

root = '/gpfs3/well/papiez/users/hri611/python/nnUnet_raw/'
new_dir = 'Dataset222_PETCT'

mask_folder = os.path.join(cropped_root, 'petct')
mask_files = [f for f in os.listdir(mask_folder) if f.endswith('.npy')]

# Process each .npy file and save it as .nii.gz
for mask_file in mask_files:
    mask_array = np.load(os.path.join(mask_folder, mask_file))

    ct_array = mask_array[0]
    pet_array = mask_array[1]

    # ct:

    nii_filename = mask_file.replace('petct.npy', '0000.nii.gz')
    nii_filepath = os.path.join(root,new_dir,'imagesTs', nii_filename)
    mask_nii = nib.Nifti1Image(ct_array, affine=np.eye(4))
    nib.save(mask_nii, nii_filepath)

    # pet:

    nii_filename = mask_file.replace('petct.npy', '0001.nii.gz')
    nii_filepath = os.path.join(root,new_dir,'imagesTs', nii_filename)
    mask_nii = nib.Nifti1Image(pet_array, affine=np.eye(4))
    nib.save(mask_nii, nii_filepath)

In [26]:
pet_array.shape

(128, 128, 128)

In [10]:

# save labels 
root = '/gpfs3/well/papiez/users/hri611/python/nnUnet_raw/'
new_dir = 'Dataset222_PETCT'

cropped_root = 'test_data'

mask_folder = os.path.join(cropped_root, 'pet_mask')
mask_files = [f for f in os.listdir(mask_folder) if f.endswith('.npy')]
mask_files

['PETCT_9902caa7ec_08132006_mask.npy',
 'PETCT_ae46202abb_02282003_mask.npy',
 'PETCT_bd76c8d299_10252002_mask.npy',
 'PETCT_176bdc5388_06222006_mask.npy',
 'PETCT_b621742469_11042004_mask.npy',
 'PETCT_d626611daf_04032003_mask.npy',
 'PETCT_9c36b318e8_11042006_mask.npy',
 'PETCT_44b95650c3_10022003_mask.npy',
 'PETCT_14929994cf_04302005_mask.npy',
 'PETCT_9aa97cf103_07182004_mask.npy',
 'PETCT_3049d4f1a4_02242002_mask.npy',
 'PETCT_4c75fa4a5d_04192001_mask.npy',
 'PETCT_21e4ffcb52_05062002_mask.npy',
 'PETCT_a87dda9193_01262003_mask.npy',
 'PETCT_ae6a37a9d6_08182005_mask.npy',
 'PETCT_c799ccc2f6_08262002_mask.npy',
 'PETCT_b899150306_08112002_mask.npy',
 'PETCT_f8de0cde56_12052005_mask.npy',
 'PETCT_f650c87621_03112005_mask.npy',
 'PETCT_d3208ff062_03142003_mask.npy',
 'PETCT_88bcc970b3_02282002_mask.npy',
 'PETCT_caae8d63f0_01262004_mask.npy',
 'PETCT_3eac8f16d4_09302001_mask.npy',
 'PETCT_1fa22c576e_10172004_mask.npy',
 'PETCT_e9feb1135e_07012005_mask.npy',
 'PETCT_38a374d01a_052420

In [6]:
## Retry getting petct images and random cropping but before preporcessing

In [32]:
list_file = 'test_imgs.txt'

cropped_root = 'test_data'

names =[]
with open(list_file) as f:
    for line in f:
        # list of files will have names of all images without file ending
        line = line.strip()
        name = line
        names.append(name)
names

['PETCT_12025abab5_11012004',
 'PETCT_f8314eb3f7_03142003',
 'PETCT_b7c1533a39_02122005',
 'PETCT_38a374d01a_05242007',
 'PETCT_456d14846b_09012003',
 'PETCT_21853fc15b_10122002',
 'PETCT_1ac497ed9d_10282004',
 'PETCT_802f19931c_06302002',
 'PETCT_21e4ffcb52_05062002',
 'PETCT_6d62e15c29_02232007',
 'PETCT_d8d9e52cd5_05072007',
 'PETCT_d3208ff062_03142003',
 'PETCT_ec581d49ef_01232003',
 'PETCT_0ea07b421b_10272001',
 'PETCT_41bd54f97a_03252007',
 'PETCT_a8a5e96821_03112005',
 'PETCT_f8de0cde56_12052005',
 'PETCT_af73b85f2c_06192003',
 'PETCT_71ac560ffe_03192007',
 'PETCT_3eac8f16d4_09302001',
 'PETCT_6a3477cd9a_01152005',
 'PETCT_44b95650c3_10022003',
 'PETCT_d32cf972af_03132003',
 'PETCT_a627fc68a1_02142003',
 'PETCT_a87dda9193_01262003',
 'PETCT_c852dfc0a5_07262002',
 'PETCT_9c36b318e8_11042006',
 'PETCT_15a205ffcc_08192005',
 'PETCT_b66ba83594_03132003',
 'PETCT_ff1451316e_03312003',
 'PETCT_389462c0ac_03032005',
 'PETCT_08212d7f6c_09112003',
 'PETCT_ae6a37a9d6_08182005',
 'PETCT_9a

In [39]:
import nibabel as nib
initial_dir = '/gpfs3/well/papiez/users/hri611/python/nnUnet_raw/Dataset219_PETCT/imagesTr'

root = '/gpfs3/well/papiez/users/hri611/python/nnUnet_raw/'
new_dir = 'Dataset222_PETCT'

list_file = 'test_imgs.txt'
cropped_root = 'test_data'

names =[]
with open(list_file) as f:
    for line in f:
        # list of files will have names of all images without file ending
        line = line.strip()
        name = line
        names.append(name)
        

for image in names[0:10]:
    pet_image_path = os.path.join(initial_dir,image+'_0001.nii.gz')
    pet_array = nib.load(pet_image_path)


In [ ]:
def Random_Crop_128_oversample(sample,random_seed = None):
    # input = sample dictionary with petct npy and label npy, returns another dictionary of numpy arrays that is cropped

    if random_seed is not None:
        np.random.seed(random_seed)
        random.seed(random_seed)
    

    image, label = sample['image'],sample['label']
    Max_C,max_H,max_W,max_D = image.shape

    if np.max(label)>0:
            # if there is some positive regions in the image, should happen 1/2 of time, make sure random cro[ includes pos point
        indices = np.argwhere(label == 1)
        random_index = np.random.choice(len(indices))
        random_point = indices[random_index] # get 4d coordinates - but how do i know they they are at the edges
        C,H,W,D = random_point[0],random_point[1],random_point[2],random_point[3]
        if H + 64 > max_H: # check if point is not at edge of cube
            H = H - 64
        elif H - 64 < 0:
            H = H + 64
        if W + 64 > max_W:
            W = W - 64
        elif W - 64 < 0:
            W = W+64
        if D + 64 > max_D:
            D = D-64
        elif D - 64 < 0:
            D = D+64
            
    else:
        H = random.randint(64, max_H - 64)
        W = random.randint(64, max_W - 64)
        D = random.randint(64, max_D - 64)
    
    image = image[...,H-64: H + 64, W-64: W + 64, D-64: D + 64] 
    label = label[...,H-64: H + 64, W-64: W + 64, D-64: D + 64] 
        
    return {'image': image, 'label': label}

In [41]:
pet_array.shape

(400, 400, 338)

In [56]:
# using raw data instead of pre-processed
initial_dir = '/gpfs3/well/papiez/users/hri611/python/nnUnet_raw/Dataset219_PETCT/imagesTr'
initial_labels_dir='/gpfs3/well/papiez/users/hri611/python/nnUnet_raw/Dataset219_PETCT/labelsTr'

root = '/gpfs3/well/papiez/users/hri611/python/nnUnet_raw/'
new_dir = 'Dataset222_PETCT'

list_file = 'test_imgs.txt'
cropped_root = 'test_data'

cropped_labels_dir = '/gpfs3/well/papiez/users/hri611/python/nnUnet_raw/Dataset222_PETCT/labelsTs_raw'
cropped_imgs_dir = '/gpfs3/well/papiez/users/hri611/python/nnUnet_raw/Dataset222_PETCT/imagesTs_raw'

names =[]
with open(list_file) as f:
    for line in f:
        # list of files will have names of all images without file ending
        line = line.strip()
        name = line
        names.append(name)

for image_name in names:
    
    pet_image_path = os.path.join(initial_dir,image_name+'_0001.nii.gz')
    ct_image_path = os.path.join(initial_dir,image_name+'_0000.nii.gz')
    label_path = os.path.join(initial_labels_dir,image_name+'.nii.gz')

    # need to take in nii image and convert to numpy
    pet_array = nib.load(pet_image_path).get_fdata()
    ct_array = nib.load(ct_image_path).get_fdata()
    label = nib.load(label_path).get_fdata()
    label_reshaped = label[np.newaxis, ...] # add an extra dimension so it works with random crop function

    image = np.stack((ct_array, pet_array), axis=0)

    sample = {'image':image,'label':label_reshaped}

    cropped_sample = Random_Crop_128_oversample(sample,random_seed=42)

    # convert to nii and save
    cropped_pet = cropped_sample['image'][1]
    cropped_ct = cropped_sample['image'][0]
    cropped_label = cropped_sample['label'][0]
    
    # resave new images in folder (need to make these folders)

    mask_nii = nib.Nifti1Image(cropped_pet, affine=np.eye(4))
    nib.save(mask_nii, os.path.join(cropped_imgs_dir,image_name+'_0001.nii.gz'))

    mask_nii = nib.Nifti1Image(cropped_ct, affine=np.eye(4))
    nib.save(mask_nii, os.path.join(cropped_imgs_dir,image_name+'_0000.nii.gz'))

    mask_nii = nib.Nifti1Image(cropped_label, affine=np.eye(4))
    nib.save(mask_nii, os.path.join(cropped_labels_dir,image_name+'.nii.gz'))
    

In [58]:
len(os.listdir(cropped_imgs_dir))

182

In [6]:
# using un cropped data (not pre-processed either)
import shutil

initial_dir = '/gpfs3/well/papiez/users/hri611/python/nnUnet_raw/Dataset219_PETCT/imagesTr'
initial_labels_dir='/gpfs3/well/papiez/users/hri611/python/nnUnet_raw/Dataset219_PETCT/labelsTr'

list_file = 'test_imgs.txt'

cropped_labels_dir = '/gpfs3/well/papiez/users/hri611/python/nnUnet_raw/Dataset222_PETCT/labelsTs_uncropped'
cropped_imgs_dir = '/gpfs3/well/papiez/users/hri611/python/nnUnet_raw/Dataset222_PETCT/imagesTs_uncropped'

names =[]
with open(list_file) as f:
    for line in f:
        # list of files will have names of all images without file ending
        line = line.strip()
        name = line
        names.append(name)

for image_name in names:
    pet_image_path = os.path.join(initial_dir,image_name+'_0001.nii.gz')
    ct_image_path = os.path.join(initial_dir,image_name+'_0000.nii.gz')
    label_path = os.path.join(initial_labels_dir,image_name+'.nii.gz')

    new_pet_image_path = os.path.join(cropped_imgs_dir,image_name+'_0001.nii.gz')
    new_ct_image_path = os.path.join(cropped_imgs_dir,image_name+'_0000.nii.gz')
    new_label_path = os.path.join(cropped_labels_dir,image_name+'.nii.gz')

    shutil.copy(pet_image_path,new_pet_image_path)
    shutil.copy(ct_image_path,new_ct_image_path)
    shutil.copy(label_path,new_label_path)



In [20]:
# crop nnunet after inference
import nibabel as nib

inferTs_uncropped = '/gpfs3/well/papiez/users/hri611/python/nnUNet_results/Dataset219_PETCT/nnUNetTrainer_1500epochs_NoMirroring__nnUNetPlans__3d_fullres/inferTs222_uncropped'
labelsTs_uncropped = '/gpfs3/well/papiez/users/hri611/python/nnUnet_raw/Dataset222_PETCT/labelsTs_uncropped'

inferTs_postcropping = '/gpfs3/well/papiez/users/hri611/python/nnUNet_results/Dataset219_PETCT/nnUNetTrainer_1500epochs_NoMirroring__nnUNetPlans__3d_fullres/inferTs222_postcropping'
labelTs_postcropping = '/gpfs3/well/papiez/users/hri611/python/nnUNet_results/Dataset219_PETCT/nnUNetTrainer_1500epochs_NoMirroring__nnUNetPlans__3d_fullres/labelsTs222_postcropping'

os.makedirs(inferTs_postcropping, exist_ok=True)
os.makedirs(labelTs_postcropping, exist_ok=True)

random_seed = 42

for image_name in os.listdir(inferTs_uncropped):
    if ".nii.gz" in image_name:
        image = nib.load(os.path.join(inferTs_uncropped,image_name)).get_fdata()
        label = nib.load(os.path.join(labelsTs_uncropped,image_name)).get_fdata()

        np.random.seed(random_seed)
        random.seed(random_seed)
        
        max_H,max_W,max_D = image.shape

        print(max_H)
        print(max_W)
        print(max_D)
    
        H = int(np.round(max_H/2))
        W = int(np.round(max_W/2))
        D = int(np.round(max_D/2))
            
        # if np.max(label)>0:
        #         # if there is some positive regions in the image, should happen 1/2 of time, make sure random cro[ includes pos point
        #     indices = np.argwhere(label == 1)
        #     random_index = np.random.choice(len(indices))
        #     random_point = indices[random_index] # get 4d coordinates - but how do i know they they are at the edges
        #     H,W,D = random_point[0],random_point[1],random_point[2]
        #     if H + 64 > max_H: # check if point is not at edge of cube
        #         H = H - 64
        #     elif H - 64 < 0:
        #         H = H + 64
        #     if W + 64 > max_W:
        #         W = W - 64
        #     elif W - 64 < 0:
        #         W = W+64
        #     if D + 64 > max_D:
        #         D = D-64
        #     elif D - 64 < 0:
        #         D = D+64
                
        # else:
        #     H = random.randint(64, max_H - 64)
        #     W = random.randint(64, max_W - 64)
        #     D = random.randint(64, max_D - 64)
        
        image = image[H-64: H + 64, W-64: W + 64, D-64: D + 64] 
        label = label[H-64: H + 64, W-64: W + 64, D-64: D + 64]

        # need to transpose images so they are in correct form
        image = np.transpose(image, (2, 1, 0))
        label = np.transpose(label, (2, 1, 0))


        # save cropped images as nifti files in new folders
        mask_nii = nib.Nifti1Image(image, affine=np.eye(4))
        nib.save(mask_nii, os.path.join(inferTs_postcropping,image_name+'.nii.gz'))
        
        mask_nii = nib.Nifti1Image(label, affine=np.eye(4))
        nib.save(mask_nii, os.path.join(labelTs_postcropping,image_name+'.nii.gz'))
        

400
400
334
400
400
326
400
400
284
400
400
326
400
400
326
400
400
326
400
400
277
400
400
284
400
400
326
400
400
284
400
400
586
400
400
326
400
400
326
400
400
326
400
400
326
400
400
326
400
400
284
400
400
284
400
400
326
400
400
326
400
400
326
400
400
326
400
400
284
400
400
284
400
400
326
400
400
326
400
400
284
400
400
265
400
400
577
400
400
347
400
400
550
400
400
284
400
400
284
400
400
312
400
400
307
400
400
284
400
400
368
400
400
284
400
400
619
400
400
326
400
400
326
400
400
338
400
400
326
400
400
284
400
400
326
400
400
326
400
400
326
400
400
284
400
400
274
400
400
619
400
400
284
400
400
284
400
400
330
400
400
326
400
400
326
400
400
326
400
400
326
400
400
200
400
400
326
400
400
497
400
400
326
400
400
368
400
400
284
400
400
326
400
400
242
400
400
284
400
400
284
400
400
535
400
400
284
400
400
327
400
400
326
400
400
576
400
400
578
400
400
628
400
400
328
400
400
284
400
400
326
400
400
326
400
400
368
400
400
326
400
400
242
400
400
342
400
400
326
400


In [13]:
labelTs_postcropping = '/gpfs3/well/papiez/users/hri611/python/nnUNet_results/Dataset219_PETCT/nnUNetTrainer_1500epochs_NoMirroring__nnUNetPlans__3d_fullres/labelsTs222_postcropping'

200.0

In [5]:
len(os.listdir(labelTs_postcropping))

91

In [68]:
        image = nib.load(os.path.join(inferTs_uncropped,image_name)).get_fdata()


In [91]:
image = '/gpfs3/well/papiez/users/hri611/python/UMIS/test_data/pet_mask/PETCT_1472967bef_12212002_mask.npy'
image = np.load(image_path)
image.shape

(2, 326, 400, 400)